In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define function to load image data from directory
def load_image_data(directory, image_size=(224, 224), batch_size=32):
    classes = ['Healthy', 'Mild', 'Severe']
    datagen = ImageDataGenerator(
        rotation_range=20,
        horizontal_flip=True,
    )

    generator = datagen.flow_from_directory(
        directory,
        target_size=image_size,
        batch_size=batch_size,
        classes=classes,
        class_mode='categorical'  # For multi-class classification
    )

    return generator

# Load image data
train_generator = load_image_data('DownsampledDataset2')
val_generator = load_image_data('PreprocessedDataset/val')

class_labels = train_generator.classes



Found 1500 images belonging to 3 classes.
Found 826 images belonging to 3 classes.


In [8]:

# Define ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features
x = base_model.output

# Global average pooling layer
x = layers.GlobalAveragePooling2D()(x)

# Classifier
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(3, activation='softmax')(x)  # Assuming 5 classes

# Create model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile model with class weights
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=val_generator,
    validation_steps=len(val_generator),
)




Epoch 1/40


KeyboardInterrupt: 

In [2]:
from keras.layers import BatchNormalization,Dense,Dropout
from keras.optimizers import Adamax
from keras.models import Model
from keras import regularizers
model_name='EfficientNetB5'
base_model=tf.keras.applications.efficientnet.EfficientNetB5(include_top=False, weights="imagenet",input_shape=(224,224,3), pooling='max') 
# Note you are always told NOT to make the base model trainable initially- that is WRONG you get better results leaving it trainable
base_model.trainable=True
x=base_model.output
x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.4, seed=123)(x)       
output=Dense(3, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
lr=.001 # start with this learning rate
model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 

In [3]:
new_history = model.fit(
    train_generator,
    epochs=20,  # Reduce the number of epochs
    verbose=1,
    validation_data=val_generator,
    validation_steps=None,
    shuffle=False,
    batch_size=16  # Reduce the batch size
)


Epoch 1/20


ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[32,28,28,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model/block3c_activation/mul_1-0-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_43709]

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight

# Define function to load image data from directory
def load_image_data(directory, image_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    generator = datagen.flow_from_directory(
        directory,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='sparse'  # For multi-class classification
    )

    return generator

# Load image data
train_generator = load_image_data('DownsampledDataset2')
val_generator = load_image_data('PreprocessedDataset/val')

class_labels = np.unique(train_generator.classes)

# Define VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(3, activation='softmax')(x)

# Create model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile model with class weights
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Save the trained model
model.save('vgg19_knee_osteoarthritis_model_3_classes.h5')

Found 1500 images belonging to 3 classes.
Found 826 images belonging to 3 classes.
Epoch 1/30
47/47 [==============================] - 30s 594ms/step - loss: 8.0701 - accuracy: 0.3280 - val_loss: 1.1012 - val_accuracy: 0.3971
Epoch 2/30
47/47 [==============================] - 28s 585ms/step - loss: 1.0989 - accuracy: 0.3133 - val_loss: 1.0979 - val_accuracy: 0.4419
Epoch 3/30
47/47 [==============================] - 28s 594ms/step - loss: 1.0988 - accuracy: 0.3193 - val_loss: 1.0974 - val_accuracy: 0.4419
Epoch 4/30
47/47 [==============================] - 28s 600ms/step - loss: 1.0987 - accuracy: 0.3200 - val_loss: 1.0977 - val_accuracy: 0.3971
Epoch 5/30
47/47 [==============================] - 29s 604ms/step - loss: 1.0989 - accuracy: 0.3120 - val_loss: 1.0974 - val_accuracy: 0.4419
Epoch 6/30
47/47 [==============================] - 29s 615ms/step - loss: 1.0988 - accuracy: 0.3247 - val_loss: 1.0986 - val_accuracy: 0.4419
Epoch 7/30
47/47 [==============================] - 29s 618